<center><font size='5'>Langton ant</font></center>
<center><font size='3'>Eric Martin, CSE, UNSW</font></center>
<center><font size='3'>COMP9021 Principles of Programming</font></center>

In [ ]:
# Does not need to be executed if
# ~/.ipython/profile_default/ipython_config.py
# exists and contains:
# c.InteractiveShell.ast_node_interactivity = 'all'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation
from IPython.display import HTML

A grid is made of cells that can change colour and be either white or red. To start with, all cells are white and Langton's ant is placed somewhere on the grid, looking North. Every second or so, it moves to the adjacent cell in the direction it is looking at. The cell it was on becomes red if it was white, and white if it was red. If the ant lands on a white cell then it rotates 90 degrees clockwise; if it lands on a red cell then it rotates 90 degrees anticlockwise.

Here are the first 18 moves, writing R for a red cell, W for a white cell that has previously been red, indicating the position of the ant with an arrow that points in the direction the ant is looking at, using

* a simple arrow in case it is on a white cell (that becomes red when the ant leaves it),
* a double arrow in case it is on a red cell (that becomes white when the ant leaves it).

        →   R↓   RR   RR   RR    RR    RR     RR    RR    RR    RR    RR    RR    RR    RR    RR    RR    RR
    ↑   R   R    R←   ⇓R   WR    WR   →WR    R↓R   RRR   RRR   RRR   RRR   RRR   RRR   RR⇐   R⇓W   RWW   RWW
                           ←    ↑R    RR     RR    R⇒    RW↓   RWR   RWR   R→R   RR⇑   RRW   RRW   R⇒W   RW↓
                                                                 ←    ↑R    RR    RR    RR    RR    RR    RR

Consider a 2-dimensional `numpy` array, say `board`, defined in such a way that for all valid indexes `y` and `x`, `board[y, x]` is the array `[y, x]`. Note that when `board` is displayed, `y` is the row number, increasing from top to bottom, and `x` is the column number, increasing from left to right: 

In [ ]:
board = np.array([[(y, x) for x in range(5)] for y in range(5)])

for y in range(5):
    print('\t'.join(str(board[y, x]) for x in range(5)))

Complex numbers can be created as literals writing `j` for $i$, that has to be preceded by an integer or a floating point literal, or thanks to the `complex` class, whose `real` and `imag` attributes return the real and imaginary parts, respectively, as floating point numbers:

In [ ]:
1j, 0 -1j, 3 + -4.21j, 0j + 6.4
complex(0, 1), complex(0, -1), complex(3, -4.21), complex(6.4, 0)
complex(3, -4.21).real, complex(3, -4.21).imag

If `location` is defined as the complex number `1 + 3j`, then `board[int(location.imag), int(location.real)]` is the value of `board` at the intersection of row number 3 and column number 1:

In [ ]:
location = 1 + 3j
board[int(location.imag), int(location.real)]

Looking West of `location` on `board` is looking at [3 0], obtained by adding -1 to `location`:

In [ ]:
direction = -1
board[int((location + direction).imag), int((location + direction).real)]

On the Cartesian plane, where the $y$ axis points up, multiplying a vector $\vec{v}$ by $i$ is rotating $\vec{v}$ by 90 degrees anticlockwise. In the current context where row numbers increase by going down, multiplying a vector $\vec{v}$ by $i$ is rotating $\vec{v}$ by 90 degrees clockwise. Hence the following code snippet rotates the unit vector that points West by 90 degrees clockwise and adds it to `location`, rotates the unit vector that points North by 90 degrees clockwise and adds it to `location`, rotates the unit vector that points East by 90 degrees clockwise and adds it to `location`, and rotates the unit vector that points South by 90 degrees clockwise and adds it to `location`; so from [3 1] we move North, then East, then South, then West:

In [ ]:
board[int(location.imag), int(location.real)]
for _ in range(4):
    direction *= 1j
    location += direction
    board[int(location.imag), int(location.real)]

Similarly, the following code snippet rotates the unit vector that points West by 90 degrees anticlockwise and adds it to `location`, rotates the unit vector that points South by 90 degrees anticlockwise and adds it to `location`, rotates the unit vector that points East by 90 degrees anticlockwise and adds it to `location`, and rotates the unit vector that points North by 90 degrees anticlockwise and adds it to `location`; so from [3 1] we move South, then East, then North, then West:

In [ ]:
board[int(location.imag), int(location.real)]
for _ in range(4):
    direction *= -1j
    location += direction
    board[int(location.imag), int(location.real)]

Let us define a larger `numpy` array for the ant to have space to explore, with 3 dimensions: the first one to refer to a row number, the second one to refer to a column number, the third one, set to 3, to record a triple of RGB values that will be set to one of white, red, or green for the cell which the ant is on. The `empty()` function from the `numpy` module allows us to define an array of a given shape, with arbitrary values of type `float64` by default, that can be changed with a second argument: 

In [ ]:
dim = 40
grid = np.empty((2 * dim + 1, 2 * dim + 1, 3), int)
grid

Let us define the triples of RBG values of interest:

In [ ]:
RED = 255, 0, 0
GREEN = 0, 255, 0
WHITE = 255, 255, 255

Let the variable `location` keep track of the ant's current location. To start with, we will place the ant at the centre of the grid under consideration, so set `grid[dim, dim]` to `GREEN`.

Let the variable `direction` keep track of the cardinal point the ant is currently facing, as one of the unit vectors 1 ("facing East"), -1 ("facing West"), $i$ ("facing South"), or $-i$ ("facing North").

We can define a `defaultdict`, say `locations`, to keep track of the cells of the grid that have been visited by the ant, using as keys the visited locations and as values:

* $i$ for a cell that became red, being white before, as the ant came on it and rotated clockwise by 90 degrees, before leaving it;
* $-i$ for a cell that became white, being red before, as the ant came on it and rotated anticlockwise by 90 degrees, before leaving it.

Also, let the default value associated with a new key, for a cell visited by the ant for the first time, be $-i$, and let -1 ("looking West") be the initial value of `direction`. If $C$ is a cell recorded as a key $K$ of `locations` and the ant visits $C$, whether for the first time or again, then we can:

* first, multiply $K$ by -1 to let $C$ take the colour it has after the ant has left it (red if $C$ had not been visited before);
* second, multiply `direction` by the resulting value ($i$ or $-i$) to compute the direction taken by the ant when it leaves $C$ (North to start with).

Let us put the initialisation code in a function:

In [ ]:
def initialise():
    global grid, locations, position, direction
    grid[:] = WHITE
    grid[dim, dim] = GREEN
    locations = defaultdict(lambda: -1j)
    position = complex(dim, dim)
    direction = -1

Let us use `matplotlib` to display the grid after it has been initialised:

In [ ]:
figure_size = 5, 5
figure = plt.figure(figsize=figure_size, dpi=192)
_ = plt.xticks([])
_ = plt.yticks([])
initialise()
state = plt.imshow(grid)

Let us use the `FuncAnimation` class from the `matplotlib.animation` module to display the ant's exploration of the grid, setting `frames` to an integer, equal to the number of frames to create.

The function to call at each frame, `simulate()`, will be passed two arguments:

* a first argument set to 0, then to 1, ..., then to the value of `frames` minus 1, to refer to each of the `frames` many frames to create;
* a second argument, `every`, given a default value of 1, to create a new frame after `every` many moves of the ant (so by default, create a frame for each move of the ant); the default value will be overwritten if `FuncAnimation` is called with `fargs` set to a 1-element tuple the value of whose unique element is the intended value of the argument `every` of `simulate()`.

The first time `simulate()` will be called, the value of `i` will be 0 and the function will return the original state of the grid; then, at each new call, the value of `i` will have increased by 1 and the function will return the state of the grid after `every` many more moves from the ant:

In [ ]:
def simulate(i, every=1):
    global grid, locations, position, direction
    if i == 0:
        state.set_data(grid)
        return state,
    for _ in range(every):
        grid[int(position.imag), int(position.real)] =\
                {-1j: RED, 1j: WHITE}[locations[position]]
        locations[position] *= -1
        direction *= locations[position]
        position += direction
        grid[int(position.imag), int(position.real)] = GREEN
    state.set_data(grid)
    return state,

To create the `FuncAnimation` object, we set the argument `init_func` to `initialise`; this function will be called before the first frame is created. We first create an animation for the first 50 moves of the ant. The `to_jshtml()` method of the `HTML` class of the `IPython.display` module can be used to play the animation within Jupyter, taking advantage of a number of options (displaying the next or previous frame, running the simulation only once or starting again from the beginning when it reaches the last frame, by default with 1000 milliseconds between 2 successive frames but the speed can be increased or decreased):

In [ ]:
HTML(animation.FuncAnimation(figure, simulate,
                             init_func=initialise, 
                             frames=50,
                             interval=1000
                            ).to_jshtml())

Let us now create 112 frames and display the state of the game every 100th move of the ant. So the last frame shows the state of the game after 11,200 moves. At this stage, a "highway" of 104 steps has emerged, that the ant's further moves will extend forever (the code is not meant to be run if the ant moves beyond the grid's boundaries; it crashes if the value of `frames` is changed to 113).

In [ ]:
HTML(animation.FuncAnimation(figure, simulate, fargs=(100,),
                             init_func=initialise, 
                             frames=112,
                             interval=200
                            ).to_jshtml())

The program [langton_ant.py](langton_ant.py) is not essentially different to the code that has been written, but uses the `pygame` module. The speed of the animation can be changed by pressing the number keys, and the program quit by pressing LeftShift Q.